In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# 基础提示链（Basic Prompt Chaining）

In [2]:
from langchain_core.prompts import PromptTemplate


# 定义提示模板
story_prompt = PromptTemplate(
    input_variables=["genre"],
    template="用 3-4 句话写一个简短的{genre}故事。"
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="用一句话概括以下故事：\n{story}"
)

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="qwen-max", temperature=1, top_p=0.8, extra_body={"enable_thinking": False}
)

# 连接提示链
def story_chain(genre):
    """根据指定的题材生成一个故事及其摘要。

    参数:
        genre (str): 要生成的故事题材类型。

    返回:
        tuple: 一个元组，包含生成的故事和其摘要。
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary

In [8]:
from IPython.display import display, Markdown

# 测试提示链
genre = "科幻"
story, summary = story_chain(genre)
display(Markdown((f"# 故事: \n{story}\n\n# 摘要: \n{summary}")))

# 故事: 
在遥远的未来，地球上的科学家们终于发明了时间旅行技术。第一次试验中，勇敢的探险家艾丽丝成功地回到了过去，却意外发现自己成为了历史书籍中的无名英雄。她留下的唯一线索是一张古老的照片，显示着一个模糊的身影站在21世纪的街头，手中握着一本未来的日记。

# 摘要: 
勇敢的探险家艾丽丝通过时间旅行回到过去，意外成为无名英雄，并留下一张手持未来日记站在21世纪街头的模糊照片作为唯一线索。

# 顺序提示（Sequential Prompting）

In [9]:
# 为每个分析步骤定义提示模板
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="识别以下文本的主要主题：\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="描述以下文本的整体语气：\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="根据以下文本，其主要主题是“{theme}”，语气为“{tone}”，请总结出关键要点：\n{text}"
)

In [10]:
def analyze_text(text):
    """对给定文本执行多步骤分析。

    参数：
        text (str)：需要分析的文本。

    返回：
        dict：包含文本主题、语气以及关键要点的字典。
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

In [13]:
# 测试顺序提示
sample_text = (
    "人工智能的迅猛发展在专家中引发了既激动又担忧的情绪。"
    "虽然 AI 有望彻底变革各行各业并改善我们的日常生活，"
    "但它也带来了关于隐私、岗位流失以及被滥用的伦理问题。"
    "当我们站在这场技术革命的门口，必须以谨慎和前瞻性的态度推进 AI 的发展，"
    "以确保其优势最大化、风险最小化。"
)

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}:\n")
    display(Markdown(value))

Theme:



这段文本的主要主题是关于人工智能（AI）发展的双面性。一方面，它强调了AI技术进步所带来的积极影响，如行业变革和生活质量的提升；另一方面，则讨论了伴随这些进步而来的潜在风险与挑战，包括隐私问题、就业市场的变化以及伦理道德上的考量。文本呼吁在推进AI技术的同时，需要采取谨慎且前瞻性的方法来平衡其利弊。简而言之，主题可以概括为“人工智能的发展及其带来的机遇与挑战”。

Tone:



这段文本的整体语气是审慎和平衡的。它既表达了对人工智能技术发展潜力的乐观期待，也明确指出了伴随而来的潜在风险与挑战。通过强调需要谨慎及前瞻性地对待AI的发展，体现了作者对于如何妥善利用这项技术、平衡利益与风险持有一种负责任的态度。整体上，这种语气旨在唤起读者对于AI技术双刃剑性质的认识，并倡导采取积极但又不失警惕的方式来推动其健康发展。

Takeaways:



关键要点总结如下：

1. **人工智能的双面性**：一方面，AI技术的发展为各行各业带来了积极的变化，能够显著改善人们的生活质量；另一方面，它也引发了包括隐私侵犯、就业市场变动以及伦理道德问题在内的多方面挑战。

2. **专家视角**：专业人士对AI的进步既感到兴奋又存在忧虑，这反映了技术本身具有极大潜力同时也伴随着潜在风险的事实。

3. **谨慎与前瞻性的态度**：鉴于AI技术可能带来的广泛影响，在推进其发展时需要采取一种平衡的方法，既要充分利用其优势促进社会进步，又要警惕并尽量减少可能出现的问题。

4. **最大化利益与最小化风险**：面对即将到来的技术革命，重要的是找到一种方式来最大化利用AI的好处，同时有效地管理和减轻任何负面影响。

# 动态提示生成（Dynamic Prompt Generation）

In [14]:
# 定义提示模板
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="简明扼要地回答以下问题：\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="根据问题“{question}”和回答“{answer}”，生成一个相关的后续问题。"
)

In [15]:
def dynamic_qa(initial_question, num_follow_ups=3):
    """进行一次动态问答对话，自动生成后续问题。

    参数：
        initial_question (str)：用于开始问答会话的初始问题。
        num_follow_ups (int)：要生成的后续问题数量。

    返回：
        list：一个包含问题与答案字典的列表。
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 是包括初始问题
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if _ < num_follow_ups:  # 最后一次不再生成后续问题
            current_question = (follow_up_prompt | llm).invoke({
                "question": current_question,
                "answer": answer
            }).content

    return qa_chain

In [16]:
# 测试动态问答系统
initial_question = "量子计算有哪些潜在应用？"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"问题{i+1}：{qa['question']}")
    print(f"回答{i+1}：{qa['answer']}\n")

问题1：量子计算有哪些潜在应用？
回答1：量子计算的潜在应用包括：
1. 大数据分析：加速处理大规模数据集。
2. 优化问题解决：如物流、调度等复杂系统的最优解搜索。
3. 化学和材料科学：模拟分子结构，发现新材料或药物。
4. 人工智能：提高机器学习模型训练速度与准确性。
5. 密码学：开发新的加密技术及破解现有加密算法。

问题2：考虑到量子计算在多个领域的潜在应用，一个相关的后续问题是：“在这些应用中，哪一项目前被认为是最接近实际应用的？请说明理由。”这样的问题可以帮助更深入地了解量子计算技术的发展现状及其对不同行业可能产生的影响。
回答2：目前被认为最接近实际应用的量子计算领域是量子化学模拟。理由如下：

1. **理论基础成熟**：量子力学本身就是描述微观粒子行为（如原子、分子）的基础理论，因此使用量子计算机来模拟这些系统具有天然优势。
2. **现有算法支持**：已经有一些特定于解决化学问题的量子算法被开发出来，比如变分量子本征解求解器(VQE)等，这些算法能够在较短时间内找到近似解。
3. **实验验证进展**：近年来，在实验室条件下利用小型量子处理器进行简单的分子模拟实验取得了初步成功，这为未来更大规模的应用打下了良好基础。
4. **市场需求明确**：制药业、材料科学等领域对于更高效准确地预测新药物或新材料性质的需求十分迫切，而量子计算正好可以提供这种能力。

综上所述，鉴于其理论上的可行性以及当前技术实现的可能性，量子化学模拟被视为短期内最有潜力实现商业价值的应用之一。

问题3：基于上述回答，一个相关的后续问题是：“量子化学模拟在哪些具体的应用场景中展现出最大的潜力？请举例说明，并讨论这些应用场景对相关行业（如制药业、材料科学等）可能带来的变革。” 这个问题有助于进一步探讨量子化学模拟的具体应用案例及其潜在影响，加深对于量子计算如何改变特定行业的理解。
回答3：量子化学模拟在多个领域展现出巨大潜力，尤其在制药业、材料科学以及能源开发等行业。下面是一些具体的应用场景及其对相关行业的潜在影响：

1. **药物设计与发现**：通过量子化学模拟，研究人员可以更准确地预测分子之间的相互作用力，从而加速新药的研发过程。例如，在抗病毒药物或抗癌药物的研究中，这种方法可以帮助科学家们更好地理解药物是如何与目标蛋白结合的，进而优化药物结构，提高其疗效和减少副作

# 错误处理与验证（Error Handling and Validation）

In [22]:
# 定义提示模板
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="生成一个与主题“{topic}”相关的四位数字。只返回该数字，不要包含任何额外文本。"
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="数字 {number} 是否真的与主题“{topic}”相关？请回答“yes”或“no”，并解释原因。"
)

In [23]:
import re

def extract_number(text):
    """从文本中提取一个四位数字。

    参数：
        text (str)：待提取数字的文本内容。

    返回：
        str 或 None：提取到的四位数字，若未找到则返回 None。
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

In [24]:
def robust_number_generation(topic, max_attempts=3):
    """生成与主题相关的四位数字，并进行验证与错误处理。

    参数：
        topic (str)：要生成数字的主题。
        max_attempts (int)：最多尝试的次数。

    返回：
        str：一个验证通过的四位数字，或错误信息。
    """
    for attempt in range(max_attempts):
        try:
            # 生成数字
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"无法从响应中提取四位数字：{response}")
            
            # 验证生成的数字是否与主题相关
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"第 {attempt + 1} 次尝试：数字 {number} 未通过验证。原因：{validation}")
        except Exception as e:
            print(f"第 {attempt + 1} 次尝试失败：{str(e)}")
    
    return "多次尝试后仍未成功生成有效数字。"

In [25]:
# 测试鲁棒数字生成逻辑
topic = "第二次世界大战"
result = robust_number_generation(topic)
print(f"主题“{topic}”的最终结果：{result}")

主题“第二次世界大战”的最终结果：1945
